# ETL Code

In [1]:
# Importing dependencies
import pandas as pd, requests
from config import WEATHER_API_KEY
import time
from datetime import datetime

In [2]:
# Reading file containing latitudes and longitudes of all states
states_df = pd.read_excel('Resources/All_States.xlsx')
states_df.head()

,state,latitude,longitude,name
0,AK,63.588753,-154.493062,Alaska
1,AL,32.318231,-86.902298,Alabama
2,AR,35.201050,-91.831833,Arkansas
3,AZ,34.048928,-111.093731,Arizona
4,CA,36.778261,-119.417932,California


In [3]:
# Selecting our 4 prefered states: AK, CA, DC and MA
pref_states = states_df[states_df['state'].isin(['DC','CA','MA', 'AK'])]
pref_states

,state,latitude,longitude,name
0,AK,63.588753,-154.493062,Alaska
4,CA,36.778261,-119.417932,California
7,DC,38.905985,-77.033418,District of Columbia
19,MA,42.407211,-71.382437,Massachusetts


In [4]:
# Creating the urls of the 4 states
# Start date set as 1606453200 or 11/27/20 5am (earliest historical data available in OpenWeather for AQI)
# End date set as 1631336400 or 9/11/21 5am
for index, row in pref_states.iterrows():
    url = "http://api.openweathermap.org/data/2.5/air_pollution/history?lat=" + str(row[1]) + "&lon=" + str(row[2]) + "&start=1606453200&end=1630468800"  + "&appid=" + WEATHER_API_KEY
    response = requests.get(url).json()
    print(url)
    # print(response)

http://api.openweathermap.org/data/2.5/air_pollution/history?lat=63.588753&lon=-154.493062&start=1606453200&end=1630468800&appid=b4f0b753cb08149f9b346a17c5fb0ff7
http://api.openweathermap.org/data/2.5/air_pollution/history?lat=36.778261&lon=-119.417932&start=1606453200&end=1630468800&appid=b4f0b753cb08149f9b346a17c5fb0ff7
http://api.openweathermap.org/data/2.5/air_pollution/history?lat=38.905985&lon=-77.033418&start=1606453200&end=1630468800&appid=b4f0b753cb08149f9b346a17c5fb0ff7
http://api.openweathermap.org/data/2.5/air_pollution/history?lat=42.407211&lon=-71.382437&start=1606453200&end=1630468800&appid=b4f0b753cb08149f9b346a17c5fb0ff7


In [5]:
# Making get requests and creating json files for 4 states
ak_data = requests.get('http://api.openweathermap.org/data/2.5/air_pollution/history?lat=63.588753&lon=-154.493062&start=1606453200&end=1630468800&appid=b4f0b753cb08149f9b346a17c5fb0ff7').json()
ma_data = requests.get('http://api.openweathermap.org/data/2.5/air_pollution/history?lat=42.407211&lon=-71.382437&start=1606453200&end=1630468800&appid=b4f0b753cb08149f9b346a17c5fb0ff7').json()
dc_data = requests.get('http://api.openweathermap.org/data/2.5/air_pollution/history?lat=38.905985&lon=-77.033418&start=1606453200&end=1630468800&appid=b4f0b753cb08149f9b346a17c5fb0ff7').json()
ca_data = requests.get('http://api.openweathermap.org/data/2.5/air_pollution/history?lat=36.778261&lon=-119.417932&start=1606453200&end=1630468800&appid=b4f0b753cb08149f9b346a17c5fb0ff7').json()

In [6]:
# Creating AK dataframe & converting time to correct format
ak = pref_states.iloc[[0]]
ak_frame = pd.json_normalize(ak_data,record_path=['list'])
ak_frame['State'] = ak['state'].iloc[0]
ak_frame['Lat'] = ak['latitude'].iloc[0]
ak_frame['Long']= ak['longitude'].iloc[0]
ak_frame['date']=[datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S') for date in ak_frame['dt']]
ak_frame.head()

,dt,main.aqi,components.co,components.no,components.no2,components.o3,components.so2,components.pm2_5,components.pm10,components.nh3,State,Lat,Long,date
0,1606453200,1,195.27,0.0,0.03,54.36,0.06,0.5,0.54,0.0,AK,63.588753,-154.493062,2020-11-27 05:00:00
1,1606456800,1,196.93,0.0,0.03,56.51,0.06,0.5,0.54,0.0,AK,63.588753,-154.493062,2020-11-27 06:00:00
2,1606460400,1,198.60,0.0,0.03,57.94,0.07,0.5,0.54,0.0,AK,63.588753,-154.493062,2020-11-27 07:00:00
3,1606464000,1,198.60,0.0,0.03,58.65,0.07,0.5,0.54,0.0,AK,63.588753,-154.493062,2020-11-27 08:00:00
4,1606467600,1,198.60,0.0,0.03,58.65,0.07,0.5,0.54,0.0,AK,63.588753,-154.493062,2020-11-27 09:00:00


In [7]:
# Creating CA dataframe & converting time to correct format
ca = pref_states.iloc[[1]]
ca_frame = pd.json_normalize(ca_data,record_path=['list'])
ca_frame['State'] = ca['state'].iloc[0]
ca_frame['Lat'] = ca['latitude'].iloc[0]
ca_frame['Long']= ca['longitude'].iloc[0]
ca_frame['date']=[datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S') for date in ca_frame['dt']]
ca_frame.head()

,dt,main.aqi,components.co,components.no,components.no2,components.o3,components.so2,components.pm2_5,components.pm10,components.nh3,State,Lat,Long,date
0,1606453200,2,283.72,0.0,7.03,48.64,0.14,16.32,18.54,3.33,CA,36.778261,-119.417932,2020-11-27 05:00:00
1,1606456800,2,270.37,0.0,5.74,44.35,0.08,11.08,13.05,2.79,CA,36.778261,-119.417932,2020-11-27 06:00:00
2,1606460400,1,257.02,0.0,4.88,40.77,0.03,6.73,8.41,2.09,CA,36.778261,-119.417932,2020-11-27 07:00:00
3,1606464000,1,250.34,0.0,4.07,43.63,0.02,4.09,5.44,1.92,CA,36.778261,-119.417932,2020-11-27 08:00:00
4,1606467600,1,240.33,0.0,3.21,48.64,0.02,2.30,3.41,1.93,CA,36.778261,-119.417932,2020-11-27 09:00:00


In [8]:
# Creating DC dataframe & converting time to correct format
dc = pref_states.iloc[[2]]
dc_frame = pd.json_normalize(dc_data,record_path=['list'])
dc_frame['State'] = dc['state'].iloc[0]
dc_frame['Lat'] = dc['latitude'].iloc[0]
dc_frame['Long']= dc['longitude'].iloc[0]
dc_frame['date']=[datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S') for date in dc_frame['dt']]
dc_frame.head()

,dt,main.aqi,components.co,components.no,components.no2,components.o3,components.so2,components.pm2_5,components.pm10,components.nh3,State,Lat,Long,date
0,1606453200,1,447.27,12.63,31.19,0.10,1.68,9.02,12.32,1.06,DC,38.905985,-77.033418,2020-11-27 05:00:00
1,1606456800,1,393.87,6.87,28.10,0.37,1.67,7.22,9.85,0.59,DC,38.905985,-77.033418,2020-11-27 06:00:00
2,1606460400,1,357.15,3.38,25.02,1.08,1.73,6.28,8.44,0.39,DC,38.905985,-77.033418,2020-11-27 07:00:00
3,1606464000,1,340.46,2.40,22.28,1.57,1.80,6.08,8.01,0.33,DC,38.905985,-77.033418,2020-11-27 08:00:00
4,1606467600,1,330.45,2.38,20.22,1.48,1.97,6.20,8.00,0.29,DC,38.905985,-77.033418,2020-11-27 09:00:00


In [9]:
# Creating MA dataframe & converting time to correct format
ma = pref_states.iloc[[3]]
ma_frame = pd.json_normalize(ma_data,record_path=['list'])
ma_frame['State'] = ma['state'].iloc[0]
ma_frame['Lat'] = ma['latitude'].iloc[0]
ma_frame['Long']= ma['longitude'].iloc[0]
ma_frame['date']=[datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S') for date in ma_frame['dt']]
ma_frame.head()

,dt,main.aqi,components.co,components.no,components.no2,components.o3,components.so2,components.pm2_5,components.pm10,components.nh3,State,Lat,Long,date
0,1606453200,1,367.17,0.53,25.36,4.65,0.31,5.44,6.72,0.14,MA,42.407211,-71.382437,2020-11-27 05:00:00
1,1606456800,1,357.15,0.29,22.96,5.01,0.26,4.91,5.87,0.08,MA,42.407211,-71.382437,2020-11-27 06:00:00
2,1606460400,1,330.45,0.09,18.34,8.14,0.22,4.14,4.91,0.06,MA,42.407211,-71.382437,2020-11-27 07:00:00
3,1606464000,1,297.07,0.04,12.85,12.52,0.21,3.40,4.01,0.04,MA,42.407211,-71.382437,2020-11-27 08:00:00
4,1606467600,1,277.04,0.02,9.17,15.74,0.24,2.91,3.41,0.04,MA,42.407211,-71.382437,2020-11-27 09:00:00


In [10]:
# Combining all 4 dataframes 
AQI_data=pd.concat([ak_frame, dc_frame, ca_frame, ma_frame])
AQI_data.columns

Index(['dt', 'main.aqi', 'components.co', 'components.no', 'components.no2',
       'components.o3', 'components.so2', 'components.pm2_5',
       'components.pm10', 'components.nh3', 'State', 'Lat', 'Long', 'date'],
      dtype='object')

In [11]:
# Renaming columns
AQI_data_Renamed = AQI_data.rename(columns = 
                {"date": "Date",
                 "State": "State",
                 "Lat": "Latitude",
                 "Long": "Longitude",
                 "main.aqi": "AQI",
                 "components.co": "CO",
                 "components.no": "NO",
                 "components.no2": "NO2",
                 "components.o3": "O3",
                 "components.so2": "SO2",
                 "components.pm2_5": "PM2_5",
                 "components.pm10": "PM10",
                 "components.nh3": "NH3",
                 "dt": "dt"})
AQI_data_Renamed.head()

,dt,AQI,CO,NO,NO2,O3,SO2,PM2_5,PM10,NH3,State,Latitude,Longitude,Date
0,1606453200,1,195.27,0.0,0.03,54.36,0.06,0.5,0.54,0.0,AK,63.588753,-154.493062,2020-11-27 05:00:00
1,1606456800,1,196.93,0.0,0.03,56.51,0.06,0.5,0.54,0.0,AK,63.588753,-154.493062,2020-11-27 06:00:00
2,1606460400,1,198.60,0.0,0.03,57.94,0.07,0.5,0.54,0.0,AK,63.588753,-154.493062,2020-11-27 07:00:00
3,1606464000,1,198.60,0.0,0.03,58.65,0.07,0.5,0.54,0.0,AK,63.588753,-154.493062,2020-11-27 08:00:00
4,1606467600,1,198.60,0.0,0.03,58.65,0.07,0.5,0.54,0.0,AK,63.588753,-154.493062,2020-11-27 09:00:00


In [12]:
# Reordering the columns
AQI_data_Renamed = AQI_data_Renamed[["Date",
                                     "State",
                                     "Latitude",
                                     "Longitude",
                                     "AQI",
                                     "CO",
                                     "NO",
                                     "NO2",
                                     "O3",
                                     "SO2",
                                     "PM2_5",
                                     "PM10",
                                     "NH3",
                                     "dt"]]
AQI_data_Renamed.head()

,Date,State,Latitude,Longitude,AQI,CO,NO,NO2,O3,SO2,PM2_5,PM10,NH3,dt
0,2020-11-27 05:00:00,AK,63.588753,-154.493062,1,195.27,0.0,0.03,54.36,0.06,0.5,0.54,0.0,1606453200
1,2020-11-27 06:00:00,AK,63.588753,-154.493062,1,196.93,0.0,0.03,56.51,0.06,0.5,0.54,0.0,1606456800
2,2020-11-27 07:00:00,AK,63.588753,-154.493062,1,198.60,0.0,0.03,57.94,0.07,0.5,0.54,0.0,1606460400
3,2020-11-27 08:00:00,AK,63.588753,-154.493062,1,198.60,0.0,0.03,58.65,0.07,0.5,0.54,0.0,1606464000
4,2020-11-27 09:00:00,AK,63.588753,-154.493062,1,198.60,0.0,0.03,58.65,0.07,0.5,0.54,0.0,1606467600


In [24]:
# Converting & saving to a CSV file for visualization in Tableau
AQI_data_Renamed.to_csv('Resources/AQI_data.csv', index=True)

# AWS RDS Code

In [14]:
# Creating dataframe for state_data_table
state_data_table_df = AQI_data_Renamed[['Date','State','Latitude',"Longitude"]]
state_data_table_df.head()

,Date,State,Latitude,Longitude
0,2020-11-27 05:00:00,AK,63.588753,-154.493062
1,2020-11-27 06:00:00,AK,63.588753,-154.493062
2,2020-11-27 07:00:00,AK,63.588753,-154.493062
3,2020-11-27 08:00:00,AK,63.588753,-154.493062
4,2020-11-27 09:00:00,AK,63.588753,-154.493062


In [15]:
# Creating dataframe for aqi_data_table
aqi_data_table_df = AQI_data_Renamed[['Date','State','AQI']]
aqi_data_table_df.head()

,Date,State,AQI
0,2020-11-27 05:00:00,AK,1
1,2020-11-27 06:00:00,AK,1
2,2020-11-27 07:00:00,AK,1
3,2020-11-27 08:00:00,AK,1
4,2020-11-27 09:00:00,AK,1


In [16]:
# Creating dataframe for pollutents_data_table
pollutants_data_table_df = AQI_data_Renamed[['Date','State','CO','NO','NO2','O3','SO2','PM2_5','PM10','NH3']]
pollutants_data_table_df.head()

,Date,State,CO,NO,NO2,O3,SO2,PM2_5,PM10,NH3
0,2020-11-27 05:00:00,AK,195.27,0.0,0.03,54.36,0.06,0.5,0.54,0.0
1,2020-11-27 06:00:00,AK,196.93,0.0,0.03,56.51,0.06,0.5,0.54,0.0
2,2020-11-27 07:00:00,AK,198.60,0.0,0.03,57.94,0.07,0.5,0.54,0.0
3,2020-11-27 08:00:00,AK,198.60,0.0,0.03,58.65,0.07,0.5,0.54,0.0
4,2020-11-27 09:00:00,AK,198.60,0.0,0.03,58.65,0.07,0.5,0.54,0.0


In [18]:
# Connecting to the AWS RDS instance and writing each DataFrame to its table

# Storing environmental variable
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://final-project-db.cr0qbwwhutcx.us-east-2.rds.amazonaws.com:5432/final-project-db"
config = {"user":"root", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password········


In [22]:
# Writing state_data_table_df to table in RDS
state_data_table_df.to_sql.(url=jdbc_url, table='state_data_table', mode=mode, properties=config)

SyntaxError: invalid syntax (<ipython-input-22-db760122d0dc>, line 2)

In [23]:
# Writing state_data_table_df to table in RDS
aqi_data_table_df.write.jdbc(url=jdbc_url, table='aqi_data_table', mode=mode, properties=config)

AttributeError: 'DataFrame' object has no attribute 'write'

In [ ]:
# Writing state_data_table_df to table in RDS
pollutents_data_table_df.write.jdbc(url=jdbc_url, table='pollutants_data_table', mode=mode, properties=config)